### Current Process
1. Read in data --> Done

2. Custom Imputation --> Done

3. Add Binary Class --> Done, Should Add Binary Class Later

4. Summary Statistics Features --> Done

5. Wrapper Functions --> Done, Need to Test Though

6. Sklearn Pipeline Categorical Features --> One Hot Encoding Done

7. Sklearn Pipeline Numerical Features --> StandardScaler Done

8. Create Lagged Features --> Done

9. Modeling --> Currently XgBoost, (Maybe Try: TensorFlow Decision Tree, TensorFlow Probability Model)

10. Model Evaluation --> Accuracy, Precision, Recall, F1, Confusion Matrix (Need to add Variable Importance Based on Variance)

11. PySpark: XGBoost Classification Feature Importance

In [ ]:
# # Need to Run These in Notebook Version For Pandas UDF
! pip install pyarrow
! pip install pandas
! pip install scikit-learn
! pip install pyspark
! pip install xgboost

In [1]:
from Input_Variables.read_vars import train_data_storage, validation_data_storage, test_data_storage, \
                                      analysis_group, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      ml_models_train_split, ml_models_test_split, model_storage_location, \
                                      time_series_lag_values_created

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Feature_Generation.lag_features import Create_Lagged_Features
from Model_Evaluation.classification_evaluation import Classification_Evalaution_Metrics
from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot

from Data_Pipeline.encoding_scaling_pipeline import Feature_Transformations

from Model_Creation.pyspark_xgboost import Create_PySpark_XGBoost

from Model_Predictions.pyspark_model_preds import Model_Predictions

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data()

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# Features Lagged Value
create_lag_features=Create_Lagged_Features()

# PySpark XGBoost Model Module
create_pyspark_xgboost=Create_PySpark_XGBoost()

# Classification Evaluation
classification_evalaution_metrics=Classification_Evalaution_Metrics()

# Model Plots Feature Importance
xgboost_classification_plot=XGBoost_Classification_Plot()

# Feature Transformations
feature_transformations=Feature_Transformations()


pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()


model_predictions=Model_Predictions()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/21 06:19:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# PySpark

### 1. PySpark: Reading In Data

#### Training

In [3]:
training_df=reading_data.read_in_pyspark_training(training_data_location=train_data_storage)
training_df.show()

+--------------------+-----+-------------------+---------------------+------------------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+-------------------+---------------------+------------------+
|8W/rpnb48OMm47W2x...|328.0|2022-01-31 17:38:00| 2022-01-31T17:38:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|331.0|2022-01-31 17:43:00| 2022-01-31T17:43:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|329.0|2022-01-31 17:48:00| 2022-01-31T17:48:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|321.0|2022-01-31 17:53:00| 2022-01-31T17:53:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|315.0|2022-01-31 17:58:00| 2022-01-31T17:58:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|313.0|2022-01-31 18:03:00| 2022-01-31T18:03:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|304.0|2022-01-31 18:08:00| 2022-01-31T18:08:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|298.0|2022-01-31 18:13:00| 2022-01-31T18:13:...|        2022-01-31|
|8W/rpnb48

#### Testing

In [4]:
testing_df=reading_data.read_in_pyspark_testing(testing_data_location=test_data_storage)
testing_df.show()

+--------------------+-----+-------------------+---------------------+------------------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+-------------------+---------------------+------------------+
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 16:59:00| 2022-02-08T16:59:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:04:00| 2022-02-08T17:04:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:09:00| 2022-02-08T17:09:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:14:00| 2022-02-08T17:14:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:19:00| 2022-02-08T17:19:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:24:00| 2022-02-08T17:24:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|277.0|2022-02-08 17:29:00| 2022-02-08T17:29:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|270.0|2022-02-08 17:34:00| 2022-02-08T17:34:...|        2022-02-08|
|8W/rpnb48

### 2. PySpark: Custom Imputation Pipeline

#### Training

In [5]:
training_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
training_custom_imputation_pipeline=pandas_sklearn_pipeline.pyspark_custom_imputation_pipeline(df=training_df, 
                                                                                               output_schema=pyspark_custom_imputation_schema,
                                                                                               analysis_group=analysis_group)

training_custom_imputation_pipeline.show(1)

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------------+--------------------+-----+
| GlucoseDisplayTime|           PatientId|Value|
+-------------------+--------------------+-----+
|2022-01-31 17:35:00|8W/rpnb48OMm47W2x...|328.0|
+-------------------+--------------------+-----+
only showing top 1 row



#### Testing

In [6]:
testing_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
testing_custom_imputation_pipeline=pandas_sklearn_pipeline.pyspark_custom_imputation_pipeline(df=testing_df, 
                                                                                               output_schema=pyspark_custom_imputation_schema,
                                                                                               analysis_group=analysis_group)

testing_custom_imputation_pipeline.show(1)

+-------------------+--------------------+---------+
| GlucoseDisplayTime|           PatientId|    Value|
+-------------------+--------------------+---------+
|2022-02-08 16:55:00|8W/rpnb48OMm47W2x...|164.20488|
+-------------------+--------------------+---------+
only showing top 1 row



### 3. PySpark: Adding Binary Labels

#### Training

In [7]:
training_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=training_custom_imputation_pipeline, 
                                                                          lower=daily_stats_features_lower, 
                                                                          upper=daily_stats_features_upper)

training_df_added_binary_labels.show(1)

+-------------------+--------------------+-----+--------+
| GlucoseDisplayTime|           PatientId|Value|y_Binary|
+-------------------+--------------------+-----+--------+
|2022-01-31 17:35:00|8W/rpnb48OMm47W2x...|328.0|       1|
+-------------------+--------------------+-----+--------+
only showing top 1 row



#### Testing

In [8]:
testing_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=testing_custom_imputation_pipeline, 
                                                                          lower=daily_stats_features_lower, 
                                                                          upper=daily_stats_features_upper)

testing_df_added_binary_labels.show(1)

+-------------------+--------------------+---------+--------+
| GlucoseDisplayTime|           PatientId|    Value|y_Binary|
+-------------------+--------------------+---------+--------+
|2022-02-08 16:55:00|8W/rpnb48OMm47W2x...|164.20488|       0|
+-------------------+--------------------+---------+--------+
only showing top 1 row



### 4. PySpark: Features: Summary Statistics

#### Training

In [9]:
training_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=training_df_added_binary_labels,
                                                                                 daily_stats_features_lower=daily_stats_features_lower,
                                                                                 daily_stats_features_upper=daily_stats_features_upper)

training_features_summary_stats.show(1)

+--------------------+-----+-------------------+-----+--------+-----+----------------+------------------+------------------+------+-----+-----+----------+----------+---------------+------------------+
|           PatientId|Chunk| GlucoseDisplayTime|Value|y_Binary|index|y_summary_binary|              Mean|           Std Dev|Median|  Min|  Max|CountBelow|CountAbove|PercentageBelow|   PercentageAbove|
+--------------------+-----+-------------------+-----+--------+-----+----------------+------------------+------------------+------+-----+-----+----------+----------+---------------+------------------+
|8W/rpnb48OMm47W2x...|    0|2022-01-31 18:25:00|289.0|       1|   11|               1|310.27272727272725|15.868780098614446| 313.0|289.0|331.0|         0|        11|            0.0|0.9166666666666666|
+--------------------+-----+-------------------+-----+--------+-----+----------------+------------------+------------------+------+-----+-----+----------+----------+---------------+---------------

#### Testing

In [10]:
testing_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=testing_df_added_binary_labels,
                                                                                 daily_stats_features_lower=daily_stats_features_lower,
                                                                                 daily_stats_features_upper=daily_stats_features_upper)

testing_features_summary_stats.show(1)

+--------------------+-----+-------------------+-----+--------+-----+----------------+-----------------+-----------------+---------+---------+-----+----------+----------+---------------+------------------+
|           PatientId|Chunk| GlucoseDisplayTime|Value|y_Binary|index|y_summary_binary|             Mean|          Std Dev|   Median|      Min|  Max|CountBelow|CountAbove|PercentageBelow|   PercentageAbove|
+--------------------+-----+-------------------+-----+--------+-----+----------------+-----------------+-----------------+---------+---------+-----+----------+----------+---------------+------------------+
|8W/rpnb48OMm47W2x...|    0|2022-02-08 17:45:00|246.0|       1|   11|               1|209.5662980513139|52.66031632565954|164.20488|164.20488|277.0|         0|         5|            0.0|0.4166666666666667|
+--------------------+-----+-------------------+-----+--------+-----+----------------+-----------------+-----------------+---------+---------+-----+----------+----------+------

### 5. PySpark: Wrapper Functions

### 6. PySpark: Lag Features

#### Training

In [11]:
training_lag_features_creation=create_lag_features.pyspark_lag_features(df=training_features_summary_stats,
                                                                       time_series_lag_values_created=time_series_lag_values_created)
training_lag_features_creation.show(5)

23/04/21 06:19:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+-----+-------------------+-----+--------+-----+----------------+------------------+------------------+------+-----+-----+----------+----------+---------------+------------------+-----------+------------------+------------------+---------+---------+---------+-------------+-------------+---------------+------------------+-----------+------------------+------------------+---------+---------+---------+-------------+-------------+---------------+------------------+
|           PatientId|Chunk| GlucoseDisplayTime|Value|y_Binary|index|y_summary_binary|              Mean|           Std Dev|Median|  Min|  Max|CountBelow|CountAbove|PercentageBelow|   PercentageAbove|value_lag_1|        mean_lag_1|     std_dev_lag_1|med_lag_1|min_lag_1|max_lag_1|cnt_bel_lag_1|cnt_abv_lag_1|perc_belw_lag_1|    perc_

#### Testing

In [12]:
testing_lag_features_creation=create_lag_features.pyspark_lag_features(df=testing_features_summary_stats,
                                                                       time_series_lag_values_created=time_series_lag_values_created)
testing_lag_features_creation.show(5)

+--------------------+-----+-------------------+---------+--------+-----+----------------+-----------------+-----------------+---------+---------+-----+----------+----------+---------------+------------------+-----------+-----------------+-----------------+---------+---------+---------+-------------+-------------+---------------+------------------+-----------+-----------------+-----------------+---------+---------+---------+-------------+-------------+---------------+------------------+
|           PatientId|Chunk| GlucoseDisplayTime|    Value|y_Binary|index|y_summary_binary|             Mean|          Std Dev|   Median|      Min|  Max|CountBelow|CountAbove|PercentageBelow|   PercentageAbove|value_lag_1|       mean_lag_1|    std_dev_lag_1|med_lag_1|min_lag_1|max_lag_1|cnt_bel_lag_1|cnt_abv_lag_1|perc_belw_lag_1|    perc_abv_lag_1|value_lag_2|       mean_lag_2|    std_dev_lag_2|med_lag_2|min_lag_2|max_lag_2|cnt_bel_lag_2|cnt_abv_lag_2|perc_belw_lag_2|    perc_abv_lag_2|
+---------------

### 7. PySpark: Sklearn Categorical Pipeline in PySpark

### 8. PySpark: Sklearn Numerical Pipeline in PySpark

In [13]:
training_numerical_stages=feature_transformations.numerical_scaling(df=training_lag_features_creation)

### 9. PySpark: XGBoost Model

In [14]:
xgboost_model=create_pyspark_xgboost.xgboost_classifier(ml_df=training_lag_features_creation,
                                                        stages=training_numerical_stages,
                                                        model_storage_location=model_storage_location)

You enabled use_gpu in spark local mode. Please make sure your local node has at least 2 GPUs
[06:19:43] task 1 got new rank 0                                    (0 + 2) / 2]
[06:19:43] task 0 got new rank 1
/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


[06:19:45] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



You enabled use_gpu in spark local mode. Please make sure your local node has at least 2 GPUs


### 10. PySpark: Cross Validation

### 11. PySpark: Model Predictions

In [15]:
testing_predictions=model_predictions.create_predictions_with_model(test_df=testing_lag_features_creation, 
                                                                    model=xgboost_model)
testing_predictions.show(10)

[06:19:51] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



+--------------------+-------------------+---------+--------+----------+--------------------+--------------------+
|           PatientId| GlucoseDisplayTime|    Value|y_binary|prediction|       rawPrediction|         probability|
+--------------------+-------------------+---------+--------+----------+--------------------+--------------------+
|8W/rpnb48OMm47W2x...|2022-02-08 17:05:00|164.20488|       0|       0.0|[3.85823082923889...|[0.97933089733123...|
|8W/rpnb48OMm47W2x...|2022-02-08 17:10:00|164.20488|       0|       0.0|[3.85823082923889...|[0.97933089733123...|
|8W/rpnb48OMm47W2x...|2022-02-08 17:15:00|164.20488|       0|       0.0|[3.85823082923889...|[0.97933089733123...|
|8W/rpnb48OMm47W2x...|2022-02-08 17:20:00|164.20488|       0|       0.0|[3.85823082923889...|[0.97933089733123...|
|8W/rpnb48OMm47W2x...|2022-02-08 17:25:00|    277.0|       1|       0.0|[3.85823082923889...|[0.97933089733123...|
|8W/rpnb48OMm47W2x...|2022-02-08 17:30:00|    270.0|       1|       1.0|[-10.903

### 12. PySpark: Model Evaluation

In [16]:
from pyspark.sql.functions import col

testing_predictions=testing_predictions \
    .withColumn("y_binary", testing_predictions["y_binary"].cast("double")) \
    .withColumn("prediction", testing_predictions["prediction"].cast("double"))

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval_accuracy = MulticlassClassificationEvaluator(labelCol="y_binary", predictionCol="prediction", metricName="accuracy")
eval_precision = MulticlassClassificationEvaluator(labelCol="y_binary", predictionCol="prediction", metricName="precisionByLabel")
eval_recall = MulticlassClassificationEvaluator(labelCol="y_binary", predictionCol="prediction", metricName="recallByLabel")
eval_f1 = MulticlassClassificationEvaluator(labelCol="y_binary", predictionCol="prediction", metricName="f1")

eval_auc = BinaryClassificationEvaluator(labelCol="y_binary", rawPredictionCol="prediction")

accuracy = eval_accuracy.evaluate(testing_predictions)
precision = eval_precision.evaluate(testing_predictions)
recall = eval_recall.evaluate(testing_predictions)
f1score = eval_f1.evaluate(testing_predictions)

auc = eval_accuracy.evaluate(testing_predictions)

[06:20:48] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[06:20:49] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[06:20:50] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [17]:
print(accuracy)
print(precision)
print(recall)
print(f1score)

print(auc)

0.9760138050043141
0.9786324786324786
0.9817090597313518
0.9760038290057009
0.9760138050043141


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, FloatType
from pyspark.mllib.evaluation import MulticlassMetrics

In [ ]:
preds_and_labels = testing_predictions.select(['prediction_label','y_binary']).withColumn('label', F.col('y_binary').cast(FloatType())).orderBy('prediction_label')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction_label','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

### 13. PySpark: XGBoost Classification Feature Importance

In [ ]:
xgboost_model.

In [ ]:
xgboost_model.stages[-1]

In [ ]:
list(zip(xgboost_model.getInputCols()))

In [ ]:
xgboost_classification_plot.read_model_plot_variance(model_storage_location=model_storage_location)

# Pandas

### Pandas: Reading in Data

In [ ]:
####### Pandas
pandas_df=reading_data.read_in_pandas()
pandas_df.head(1)

### Pandas: Custom Imputation Pipeline

In [ ]:
test_pat=pandas_df[pandas_df['PatientId']=='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=']

In [ ]:
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

In [ ]:
####### Pandas
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

### Pandas: Aggregate Data at Level

### Pandas: Adding Binary Labels

In [ ]:
pandas_df_added_binary_labels=create_binary_labels.pandas_binary_labels(df=pandas_custom_imputation_pipeline, 
                                                                        lower=daily_stats_features_lower, 
                                                                        upper=daily_stats_features_upper)
pandas_df_added_binary_labels.head(1)

### Pandas: Features: Summary Statistics

In [ ]:
pandas_features_summary_stats=summary_stats_features.pandas_compressDailyValues(data=pandas_df_added_binary_labels, 
                                                                                lower=daily_stats_features_lower, 
                                                                                upper=daily_stats_features_upper)
pandas_features_summary_stats.head(1)

### Pandas: Wrapper Functions

### Pandas: Sklearn Categorical Pipeline in Pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
df=pandas_features_summary_stats[['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate', 'inserted', 
        'missing', 'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow', 'AreaAbove']]

In [ ]:
df.head()

In [ ]:
for patient_id in df['PatientId'].unique():
    # Categorical Features
    categorical_features=['inserted', 'missing']
    categorical_transformer=Pipeline([('imputer_cat', SimpleImputer(strategy='constant', fill_value=np.nan)),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor_2=ColumnTransformer([('categorical', categorical_transformer, categorical_features)],
                                    remainder = 'passthrough')

    cat_pipe_pipeline=Pipeline([('preprocessing_2', preprocessor_2)])

    transformed_data1=cat_pipe_pipeline.fit_transform(df)

    transformed_data_df=pd.DataFrame(transformed_data1)

    transformed_data_df['combine_inserted']=transformed_data_df[[0,1]].values.tolist()
    transformed_data_df['combine_missing']=transformed_data_df[[2,3]].values.tolist()
    transformed_data_df=transformed_data_df.drop(transformed_data_df.iloc[:, 0:4],axis = 1)

    transformed_data_df.columns=['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate', 
                                    'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow', 
                                    'AreaAbove', 'inserted', 'missing']

In [ ]:
transformed_data_df.columns

In [ ]:
####### Pandas
pandas_custom_categorical_pipeline=pandas_sklearn_pipeline.pandas_transform_categorical_features(df=pandas_features_summary_stats)
pandas_custom_categorical_pipeline.head(1)

### Pandas: Sklearn Numerical Pipeline in Pandas

In [ ]:
####### Pandas
pandas_custom_numerical_pipeline=pandas_sklearn_pipeline.pandas_transform_numerical_features(df=pandas_custom_categorical_pipeline)
pandas_custom_numerical_pipeline.head(1)

### Pandas: Models